In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
from standard_functions import getAPI

Do not ever put API's in your repositories. I have mine in a local file and call it here.

In [2]:
myApi = getAPI()
lol_watcher = LolWatcher(myApi)

region = 'na1'
match_id = 'NA1_4286956599'

I use these functions to get the match_id.

In [3]:
def get_puuid(name):
    """
    Input is summoner name. Outputs players puuid.
    """
    return lol_watcher.summoner.by_name(region, name)['puuid']
def get_matchlist(name):
    """
    Input is name. Output is list of matches.
    """
    return lol_watcher.match.matchlist_by_puuid(region, get_puuid(name), count=100, type='ranked')
def get_match(match_id):
    return lol_watcher.match.by_id(region, match_id)

These functions were a bit harder to come up with. Although each individual one is pretty simple, going through and figuring out how to summarize each game to a row can be tricky. 

In [29]:
def get_DataFrame(match_id):
    match = get_match(match_id)
    data = get_avgs(match)
    data.update
def get_duration(match):
    return {'gameDuration' : match['info']['gameDuration']}

def get_team_objectives(match):
    obj = match['info']['teams'][0]['objectives']
    new = {}
    for k in obj:
        for key in obj[k]:
            new[k+'_'+key] = obj[k][key]
    return new

def get_win(match):
    return {'win' : match['info']['teams'][0]['win']}
def get_avgs(match):
    avg_keys = ['kills', 'deaths', 'assists', 'champLevel', 'champExperience', 'doubleKills', 'tripleKills', 'quadraKills', 
            'pentaKills','consumablesPurchased', 'damageDealtToBuildings',
            'damageDealtToObjectives', 'damageSelfMitigated', 'detectorWardsPlaced', 'timePlayed',
            'goldEarned', 'goldSpent', 'inhibitorKills', 'itemsPurchased', 'largestKillingSpree', 'killingSprees',
            'longestTimeSpentLiving', 'magicDamageDealt','magicDamageDealtToChampions', 'magicDamageTaken',
            'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken', 'totalDamageDealt',
            'totalDamageDealtToChampions', 'totalDamageShieldedOnTeammates', 'totalDamageTaken', 
            'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken',
            'totalHeal', 'totalHealsOnTeammates', 'totalMinionsKilled', 'totalTimeCCDealt', 'totalTimeSpentDead',
            'turretKills', 'turretsLost', 'neutralMinionsKilled', 'objectivesStolen', 'objectivesStolenAssists', 
            'visionScore','visionWardsBoughtInGame', 'wardsKilled', 'wardsPlaced']
    new = {}
    for i in match['info']['participants']:
        if i['teamId'] == 100:
            for j in avg_keys:
                try:
                    new['avg_'+j] += i[j]
                except:
                    new['avg_'+j] = i[j]
    new.update((k, round(v * 0.2, 3)) for k,v in new.items())
    return new

In [39]:
match = get_match(match_id)
data = get_team_objectives(match)
data.update(get_win(match))
data.update(get_duration(match))
data.update(get_avgs(match))
data = pd.DataFrame([data])

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 63 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   baron_first                         1 non-null      bool   
 1   baron_kills                         1 non-null      int64  
 2   champion_first                      1 non-null      bool   
 3   champion_kills                      1 non-null      int64  
 4   dragon_first                        1 non-null      bool   
 5   dragon_kills                        1 non-null      int64  
 6   inhibitor_first                     1 non-null      bool   
 7   inhibitor_kills                     1 non-null      int64  
 8   riftHerald_first                    1 non-null      bool   
 9   riftHerald_kills                    1 non-null      int64  
 10  tower_first                         1 non-null      bool   
 11  tower_kills                         1 non-null   

In [16]:
match['info']['participants'][0].keys()

dict_keys(['gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'participants', 'platformId', 'queueId', 'teams', 'tournamentCode'])

In [28]:
match['info']['participants'][0]['challenges']

{'12AssistStreakCount': 0,
 'abilityUses': 208,
 'acesBefore15Minutes': 0,
 'alliedJungleMonsterKills': 2,
 'baronTakedowns': 1,
 'blastConeOppositeOpponentCount': 0,
 'bountyGold': 0,
 'buffsStolen': 0,
 'completeSupportQuestInTime': 0,
 'controlWardTimeCoverageInRiverOrEnemyHalf': 0.18260364271342508,
 'controlWardsPlaced': 2,
 'damagePerMinute': 372.0216369218776,
 'damageTakenOnTeamPercentage': 0.1915849203921449,
 'dancedWithRiftHerald': 0,
 'deathsByEnemyChamps': 8,
 'dodgeSkillShotsSmallWindow': 44,
 'doubleAces': 0,
 'dragonTakedowns': 0,
 'earliestBaron': 1382.9942794950232,
 'earlyLaningPhaseGoldExpAdvantage': -0.19972527027130127,
 'effectiveHealAndShielding': 0,
 'elderDragonKillsWithOpposingSoul': 0,
 'elderDragonMultikills': 0,
 'enemyChampionImmobilizations': 34,
 'enemyJungleMonsterKills': 0,
 'epicMonsterKillsNearEnemyJungler': 0,
 'epicMonsterKillsWithin30SecondsOfSpawn': 0,
 'epicMonsterSteals': 0,
 'epicMonsterStolenWithoutSmite': 0,
 'flawlessAces': 0,
 'fullTeamTa

In [27]:
avg_keys = ['kills', 'deaths', 'assists', 'champLevel', 'champExperience', 'doubleKills', 'tripleKills', 'quadraKills', 
            'pentaKills','consumablesPurchased', 'damageDealtToBuildings',
            'damageDealtToObjectives', 'damageSelfMitigated', 'detectorWardsPlaced', 'timePlayed',
            'goldEarned', 'goldSpent', 'inhibitorKills', 'itemsPurchased', 'largestKillingSpree', 'killingSprees',
            'longestTimeSpentLiving', 'magicDamageDealt','magicDamageDealtToChampions', 'magicDamageTaken',
            'physicalDamageDealt', 'physicalDamageDealtToChampions', 'physicalDamageTaken', 'totalDamageDealt',
            'totalDamageDealtToChampions', 'totalDamageShieldedOnTeammates', 'totalDamageTaken', 
            'trueDamageDealt', 'trueDamageDealtToChampions', 'trueDamageTaken',
            'totalHeal', 'totalHealsOnTeammates', 'totalMinionsKilled', 'totalTimeCCDealt', 'totalTimeSpentDead',
            'turretKills', 'turretsLost', 'neutralMinionsKilled', 'objectivesStolen', 'objectivesStolenAssists', 
            'visionScore','visionWardsBoughtInGame', 'wardsKilled', 'wardsPlaced']
new = {}
for i in match['info']['participants']:
    if i['teamId'] == 100:
        for j in avg_keys:
            try:
                new['avg_'+j] += i[j]
            except:
                new['avg_'+j] = i[j]
new.update((k, round(v * 0.2, 3)) for k,v in new.items())
new

{'avg_kills': 8.2,
 'avg_deaths': 12.2,
 'avg_assists': 6.6,
 'avg_champLevel': 16.0,
 'avg_champExperience': 15302.8,
 'avg_doubleKills': 1.2,
 'avg_tripleKills': 0.0,
 'avg_quadraKills': 0.0,
 'avg_pentaKills': 0.0,
 'avg_consumablesPurchased': 3.8,
 'avg_damageDealtToBuildings': 2657.8,
 'avg_damageDealtToObjectives': 9446.4,
 'avg_damageSelfMitigated': 24171.2,
 'avg_detectorWardsPlaced': 0.8,
 'avg_timePlayed': 1905.0,
 'avg_goldEarned': 13615.2,
 'avg_goldSpent': 12931.0,
 'avg_inhibitorKills': 0.0,
 'avg_itemsPurchased': 23.0,
 'avg_largestKillingSpree': 2.4,
 'avg_killingSprees': 1.6,
 'avg_longestTimeSpentLiving': 430.4,
 'avg_magicDamageDealt': 38503.8,
 'avg_magicDamageDealtToChampions': 7567.6,
 'avg_magicDamageTaken': 4398.8,
 'avg_physicalDamageDealt': 76610.2,
 'avg_physicalDamageDealtToChampions': 10587.4,
 'avg_physicalDamageTaken': 21295.2,
 'avg_totalDamageDealt': 130064.8,
 'avg_totalDamageDealtToChampions': 23560.4,
 'avg_totalDamageShieldedOnTeammates': 0.0,
 'avg

In [39]:
summoners = ['Zealfire', 'SmurfHD', 'DumbAmerican', 'About 10 Inches', 'Exprience', 'Leaders1', 
             'Kehlee', 'KybitOW', 'DodoMuncher', 'Mango Bleach']

In [4]:
get_matchlist('Zealfire')

['NA1_4348171334',
 'NA1_4289259129',
 'NA1_4287742329',
 'NA1_4287694201',
 'NA1_4287648113',
 'NA1_4286956599',
 'NA1_4286785334',
 'NA1_4286667294',
 'NA1_4285375587',
 'NA1_4285360882',
 'NA1_4285334783',
 'NA1_4285247595',
 'NA1_4284641450',
 'NA1_4284573748',
 'NA1_4282617016',
 'NA1_4282651208',
 'NA1_4282568419',
 'NA1_4282560428',
 'NA1_4279275380',
 'NA1_4279290874',
 'NA1_4272162489',
 'NA1_4270474097',
 'NA1_4270373912',
 'NA1_4270325535',
 'NA1_4270310445',
 'NA1_4269678873',
 'NA1_4269623819',
 'NA1_4265939472',
 'NA1_4265052963',
 'NA1_4265007120',
 'NA1_4265011466',
 'NA1_4264596019',
 'NA1_4264558755',
 'NA1_4263428731',
 'NA1_4262510186',
 'NA1_4262424973',
 'NA1_4259122335',
 'NA1_4259076750',
 'NA1_4258616238',
 'NA1_4257716745',
 'NA1_4251471488',
 'NA1_4248601449',
 'NA1_4248509505',
 'NA1_4247935338',
 'NA1_4243646495',
 'NA1_4242625676',
 'NA1_4242377491',
 'NA1_4242051531',
 'NA1_4241728549',
 'NA1_4241732889',
 'NA1_4240856758',
 'NA1_4240829863',
 'NA1_423986

In [6]:
pd.DataFrame([lol_watcher.match.by_id(region, 'NA1_4205025589')['info']])

,gameCreation,gameDuration,gameEndTimestamp,gameId,gameMode,gameName,gameStartTimestamp,gameType,gameVersion,mapId,participants,platformId,queueId,teams,tournamentCode
0,1643938709000,1555,1643940333289,4205025589,CLASSIC,teambuilder-match-4205025589,1643938777865,MATCHED_GAME,12.3.421.5967,11,"[{'assists': 3, 'baronKills': 0, 'bountyLevel'...",NA1,420,"[{'bans': [{'championId': 223, 'pickTurn': 1},...",


In [ ]:
pd.DataFrame([lol_watcher.match.by_id(region, 'NA1_NA1_4205025589')['info']['participants'][0]])

In [ ]:
data = lol_watcher.league.entries(region, queue='RANKED_SOLO_5x5', tier='GOLD', division='II', page=500)
len(data)

In [25]:
lol_watcher.match.by_id(region, 'NA1_4205025589')['info'].keys()

dict_keys(['gameCreation', 'gameDuration', 'gameEndTimestamp', 'gameId', 'gameMode', 'gameName', 'gameStartTimestamp', 'gameType', 'gameVersion', 'mapId', 'participants', 'platformId', 'queueId', 'teams', 'tournamentCode'])

In [26]:
lol_watcher.match.by_id(region, 'NA1_4205025589')['info']['teams'][0]

{'bans': [{'championId': 223, 'pickTurn': 1},
  {'championId': 25, 'pickTurn': 2},
  {'championId': 84, 'pickTurn': 3},
  {'championId': 81, 'pickTurn': 4},
  {'championId': 38, 'pickTurn': 5}],
 'objectives': {'baron': {'first': False, 'kills': 0},
  'champion': {'first': False, 'kills': 20},
  'dragon': {'first': False, 'kills': 0},
  'inhibitor': {'first': False, 'kills': 0},
  'riftHerald': {'first': False, 'kills': 0},
  'tower': {'first': False, 'kills': 2}},
 'teamId': 100,
 'win': False}

In [ ]:
lol_watcher.match.by_id(region, 'NA1_4205025589')['info']['teams'][0]['objectives'].keys()

In [ ]:
pd.DataFrame(lol_watcher.match.by_id(region, 'NA1_4205025589')['info']['teams'][0]['objectives'])

In [ ]:
pd.DataFrame(lol_watcher.match.timeline_by_match(region, 'NA1_4205025589')['info']['frames'])

In [ ]:
pd.DataFrame.rename()

In [50]:
obj = lol_watcher.match.by_id(region, match_id)['info']['teams'][0]['objectives']

In [51]:
obj

{'baron': {'first': True, 'kills': 1},
 'champion': {'first': True, 'kills': 41},
 'dragon': {'first': False, 'kills': 0},
 'inhibitor': {'first': False, 'kills': 0},
 'riftHerald': {'first': True, 'kills': 1},
 'tower': {'first': False, 'kills': 5}}